# ARIMAモデルのインタラクティブ・シミュレーション

本ノートブックでは、**ARIMA(p, d, q) モデル** の挙動をインタラクティブに観察します。

自己回帰項 (AR)、和分項 (I: 差分)、移動平均項 (MA) のパラメータを変更することで、
生成される時系列データや、自己相関関数 (ACF)、偏自己相関関数 (PACF) がどのように変化するかを確認してください。

## 観察のポイント
1. **AR(p) の挙動**:
   - AR係数（phi）を大きくすると、過去の値への依存が強まります。
   - ACFは徐々に減衰し、PACFはラグ $p$ でカットオフ（切断）します。
2. **MA(q) の挙動**:
   - MA係数（theta）を変更すると、過去の誤差の影響が変わります。
   - PACFは徐々に減衰し、ACFはラグ $q$ でカットオフします。
3. **非定常性 (d)**:
   - 差分階数 $d$ を1以上にすると、データはトレンドを持ち、非定常になります（ランダムウォークなど）。
   - $d=0$ の場合、データは平均の周りを振動する定常過程になります（係数が定常条件を満たす場合）。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima_process import ArmaProcess
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import ipywidgets as widgets
from IPython.display import display

# 日本語フォントの設定
import matplotlib.font_manager as fm
font_path = '../fonts/ipaexg.ttf'
fm.fontManager.addfont(font_path)
plt.rcParams['font.family'] = 'IPAexGothic'

%matplotlib inline

def plot_arima_interactive(ar1, ar2, ma1, ma2, d, n_samples=200):
    # AR係数とMA係数の設定
    # statsmodelsでは、AR項は左辺に移項した形 (1 - phi*L)y = ... なので符号を反転させる必要がある
    # しかし、ユーザーの直感的には y_t = phi * y_{t-1} が自然なので、
    # ここでは入力された正の係数を statsmodels 用に負に変換する。
    ar_params = np.array([1, -ar1, -ar2])
    ma_params = np.array([1, ma1, ma2])
    
    # ARMAプロセスの生成
    # ar1, ar2 が大きすぎると非定常になる可能性があるが、ここではそのまま通す
    try:
        process = ArmaProcess(ar_params, ma_params)
        sample = process.generate_sample(nsample=n_samples)
        
        # 差分 (Integration) の適用
        if d > 0:
            # 累積和をとることで積分過程にする（簡易的な実装）
            for _ in range(d):
                sample = np.cumsum(sample)
                
        # プロット
        fig, axes = plt.subplots(1, 3, figsize=(18, 5))
        
        # 1. 時系列プロット
        axes[0].plot(sample)
        axes[0].set_title(f"時系列データ (AR=[{ar1}, {ar2}], MA=[{ma1}, {ma2}], d={d})")
        axes[0].set_xlabel("Time")
        axes[0].set_ylabel("Value")
        axes[0].grid(True)
        
        # 定常性のチェック（AR部分の根が単位円外にあるか）
        if not process.isstationary and d == 0:
             axes[0].text(0.05, 0.9, "※非定常 (AR係数が大)", transform=axes[0].transAxes, color="red", fontweight="bold")

        # 2. 自己相関 (ACF)
        # 非定常データでACFを計算すると減衰が遅いが、ここではそのまま表示する
        plot_acf(sample, lags=20, ax=axes[1], title="自己相関 (ACF)")
        axes[1].grid(True)

        # 3. 偏自己相関 (PACF)
        try:
            plot_pacf(sample, lags=20, ax=axes[2], title="偏自己相関 (PACF)", method='ywm')
        except:
            axes[2].text(0.5, 0.5, "PACF計算エラー\n(定常性なし)", ha='center')
        axes[2].grid(True)
        
        plt.tight_layout()
        plt.show()
        
    except Exception as e:
        print(f"Error: {e}")

# ウィジェットの作成
style = {'description_width': 'initial'}

w_ar1 = widgets.FloatSlider(value=0.5, min=-1.5, max=1.5, step=0.1, description='AR(1) phi1:', style=style)
w_ar2 = widgets.FloatSlider(value=0.0, min=-1.0, max=1.0, step=0.1, description='AR(2) phi2:', style=style)
w_ma1 = widgets.FloatSlider(value=0.0, min=-1.5, max=1.5, step=0.1, description='MA(1) theta1:', style=style)
w_ma2 = widgets.FloatSlider(value=0.0, min=-1.0, max=1.0, step=0.1, description='MA(2) theta2:', style=style)
w_d = widgets.IntSlider(value=0, min=0, max=2, description='差分 d:', style=style)

ui = widgets.VBox([
    widgets.HBox([w_ar1, w_ar2]),
    widgets.HBox([w_ma1, w_ma2]),
    w_d
])

out = widgets.interactive_output(plot_arima_interactive, 
                                 {'ar1': w_ar1, 'ar2': w_ar2, 
                                  'ma1': w_ma1, 'ma2': w_ma2, 
                                  'd': w_d})

display(ui, out)

## AR(1)モデルと回帰分析の視覚的理解

AR(1)モデル $y_t = \phi y_{t-1} + \epsilon_t$ は、**「現在の値 $y_t$ を、1つ前の値 $y_{t-1}$ で回帰する（説明する）」** モデルです。
これを視覚的に確認するために、時系列データのプロットと同時に、横軸に $y_{t-1}$、縦軸に $y_t$ をとった**散布図（Lag Plot）**を表示します。

AR係数 $\phi$ を変化させると、散布図上の点の分布の「傾き」や相関の強さがどのように変わるか観察してください。

In [ ]:
from sklearn.linear_model import LinearRegression

def plot_ar1_regression(phi, n_samples=200):
    # AR(1)データの生成
    # y_t = phi * y_{t-1} + epsilon
    # statsmodelsのArmaProcessを使うと符号反転が必要だが、
    # ここではシンプルに直接生成する（初期値の影響などを排除しやすい）
    np.random.seed(42)
    epsilon = np.random.normal(0, 1, n_samples)
    y = np.zeros(n_samples)
    
    # 定常分布からのサンプリングを初期値とする（定常の場合）
    if abs(phi) < 1:
        y[0] = np.random.normal(0, 1 / np.sqrt(1 - phi**2))
    else:
        y[0] = 0

    for t in range(1, n_samples):
        y[t] = phi * y[t-1] + epsilon[t]
        
    # ラグデータの作成
    y_current = y[1:]
    y_lag1 = y[:-1]
    
    # 線形回帰（確認用）
    lr = LinearRegression()
    lr.fit(y_lag1.reshape(-1, 1), y_current)
    y_pred = lr.predict(y_lag1.reshape(-1, 1))
    
    # プロット
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    # 1. 時系列プロット
    axes[0].plot(y)
    axes[0].set_title(f"AR(1) 時系列 (phi={phi})")
    axes[0].set_xlabel("Time")
    axes[0].set_ylabel("Value")
    axes[0].grid(True)
    
    # 2. 散布図 (Lag Plot)
    axes[1].scatter(y_lag1, y_current, alpha=0.5, label="Data")
    axes[1].plot(y_lag1, y_pred, color='red', linewidth=2, label=f"Regression Line (Slope={lr.coef_[0]:.2f})")
    axes[1].set_title(f"散布図: $y_{{t-1}}$ vs $y_t$")
    axes[1].set_xlabel("$y_{t-1}$ (1期前の値)")
    axes[1].set_ylabel("$y_t$ (現在の値)")
    axes[1].legend()
    axes[1].grid(True)
    
    # 補助線
    min_val = min(y.min(), y_lag1.min())
    max_val = max(y.max(), y_lag1.max())
    axes[1].plot([min_val, max_val], [min_val, max_val], color='gray', linestyle='--', alpha=0.5, label="y=x")

    plt.tight_layout()
    plt.show()

# ウィジェット
w_phi_reg = widgets.FloatSlider(value=0.7, min=-1.1, max=1.1, step=0.1, description='phi:', style=style)

display(widgets.interactive(plot_ar1_regression, phi=w_phi_reg))

## 任意次数のAR(p)モデルとPACFの関係

ここでは、より高次の AR($p$) モデルにおける、偏自己相関関数 (PACF) の挙動を確認します。
理論的には、AR($p$) 過程の PACF は、ラグ $p$ までは有意な値を持ち、ラグ $p+1$ 以降はゼロになります（カットオフ）。

以下のスライダーで次数 $p$ を変更し、PACFがどのように変化するかを観察してください。

※ 係数は、定常性を満たすように内部で自動的に設定されます（減衰パターンなど）。

In [ ]:
def plot_ar_p_interactive(p, n_samples=500):
    # 係数の自動設定（簡易的）
    # phi_k = 0.6 * (0.8)^(k-1) * (-1)^(k-1) みたいな減衰振動パターンを与える
    # これにより、そこそこ定常性を保ちつつ、高次まで係数がある状態を作る
    
    if p == 0:
        ar_coeffs = []
    else:
        # 少し減衰させつつ、符号を交互にするなどして特徴を出す
        # 係数の絶対値の和が1を超えると非定常になりやすいので調整
        # ここでは単調減衰パターンを採用
        signs = np.ones(p)
        # signs[1::2] = -1 # 符号交互の場合
        
        # 減衰係数
        decay = 0.6
        ar_coeffs = [decay**(i+1) * signs[i] for i in range(p)]
        
        # 係数の総和チェック（簡易的な定常性確保）
        if np.sum(np.abs(ar_coeffs)) >= 1.0:
             ar_coeffs = np.array(ar_coeffs) / (np.sum(np.abs(ar_coeffs)) + 0.1)

    # statsmodels用のARパラメータ (1, -phi1, -phi2, ...)
    ar_params = np.r_[1, -np.array(ar_coeffs)]
    ma_params = np.array([1])
    
    try:
        process = ArmaProcess(ar_params, ma_params)
        sample = process.generate_sample(nsample=n_samples)
        
        # 係数の表示
        print(f"設定されたAR係数 (phi_1 ... phi_{p}):")
        print([f"{c:.3f}" for c in ar_coeffs])
        
        if not process.isstationary:
            print("※ 注意: 生成されたプロセスは非定常の可能性があります。")

        fig, axes = plt.subplots(1, 3, figsize=(18, 5))
        
        # 1. 時系列
        axes[0].plot(sample)
        axes[0].set_title(f"AR({p}) 時系列")
        axes[0].set_xlabel("Time")
        axes[0].grid(True)

        # 2. ACF
        plot_acf(sample, lags=20, ax=axes[1], title="自己相関 (ACF)")
        axes[1].grid(True)

        # 3. PACF
        plot_pacf(sample, lags=20, ax=axes[2], title="偏自己相関 (PACF)", method='ywm')
        # カットオフの期待位置を表示
        if p > 0:
            axes[2].axvline(x=p, color='red', linestyle='--', alpha=0.5, label=f'Lag {p}')
            axes[2].legend()
        axes[2].grid(True)
        
        plt.tight_layout()
        plt.show()
        
    except Exception as e:
        print(f"Error: {e}")

# ウィジェット
w_p = widgets.IntSlider(value=1, min=0, max=10, step=1, description='次数 p:', style=style)

display(widgets.interactive(plot_ar_p_interactive, p=w_p))